In [1]:
pwd

'/home/aallali/OCR_projects/GDC'

In [2]:
import time
import os
import random
import cv2
import json
from pathlib import Path

import numpy as np
from Preprocessing import Preprocessing
from Handwriting_gen import Generation

##### Create instances of relevant classes
From Preprocessing and Handwriting_gen, we use "Preprocessing" class to preprocess and adjust the images we're handling, they can be images of handwritten characters or handwritten text (string).<p>
From Handwriting_gen, we use "Generation" class to fetch, handle and process images of single characters in order to produce a set of images containing handwritten text

In [ ]:
preprocessing= Preprocessing()
generation= Generation()

Then we set the necessary attributes of each class (For more info, check out the init() function of the class)

In [5]:
source_dir= "Data/HTR/Chars74K_dataset/English/Hnd/Img/"

generation.source_dir= source_dir
generation.special_char_dir= "Data/HTR/special_characters/"

generation.dilation_kernel = np.ones((2,2), np.uint8)

generation.charW= 15
generation.charH= 25
generation.char_capital_W= 25
generation.char_capital_H= 35

#### Generate handwritten text images for text recognition

For a text recognition task, we often need two inputs:<p>
- A set of images containing text to recognize.
- list of strings corresponding to text inside these images (ground truth text).<p>
 In our case, we'll generate images containing handwritten text and text (.txt) file that contains the strings

In [6]:
def ascii_seq_to_string(ascii_seq):
    """
    Inputs:
    - ascii_seq: ascii numbers array corresponding to an array of 
                characters (e.g. [97, 90, 53] <-> ['a', 'Z', '5'])
    Outputs:
    - string: a string that assembles the characters corresponding to 
    the ASCII sequence (e.g. ascii_seq= [97, 90, 53] -> string= 'aZ5')
    """
    
    string= ""
    for n in ascii_seq:
        string= string+ str(chr(n))

    return string

A path named "saving_dir", in which the generated images can be saved, should be specified. A text file will be created in the parent folder of this file.<p>
Also, the size of generated images dataset should be specified

In [10]:
saving_dir= "Data/HTR/Generated_sequences/text_recognition/train/"
gt_txt_file = open(os.path.join(str(Path(saving_dir).parent), 'train_gt.txt'), "w+")
dataset_size= 10000

Handwritten text dataset generation is ready to be launched

In [11]:
j=0

for i in range(dataset_size):
    #1 element capital letters sequence (A->Z)
    seq1= random.choices(range(65, 91), k=1)

    #2 element capital letters sequence (A->Z)
    seq2= random.choices(range(65, 91), k=2)

    #1 element letters sequence (a->z)
    seq3= random.choices(range(97, 122), k=1)

    #2 element letters sequence (a->z)
    seq4= random.choices(range(97, 122), k=2)
    for seq in [seq1, seq2, seq3, seq4]:

        string = ascii_seq_to_string(seq)
        try:
            string_img= generation.generate_sentence_image(string)
            cv2.imwrite(os.path.join(saving_dir, f"Generated_img_{i+j}.jpg"), string_img)
            gt_txt_file.write(f"Generated_img_{i+j}.jpg\t{string}\n")
        except:
            continue

        j+=1

gt_txt_file.close()

#### Generate handwritten text images for text segmentation
For an image segmentation task, we often need two inputs:<p>
- A set of images containing text to recognize.
- list of annotations corresponding to the regions of interest in each image, the annotations should follow a certain format (COCO, VOC XML, ...), to be determined by the user.<p>
 In our case, we'll generate handwritten text images and a json file containing the annotations in COCO format

In [6]:
generation.dilation_kernel = np.ones((2,2),np.uint8)
generation.keyword= ".png"

generation.charW= 25
generation.charH= 35
generation.char_capital_W= 25
generation.char_capital_H= 35

In [9]:
saving_dir= "Data/HTR/Generated_sequences/test_annotated_images/"
json_file= 'test_coco_annotations.json'

len_dataset= 2000

img_id=0
letter_id=0
coco_dict={}
coco_dict["licenses"]= []
coco_dict["info"]= {}
coco_dict["images"]= []
coco_dict["annotations"]= []

coco_dict["categories"]= [{"supercategory": "",
                            "id": 1,
                            "keypoints": [],
                            "name": "text"}
                          ]

for i in range(len_dataset//7):
    # Cable Name in ASCII (3letters+ 4numbers + 3letters)
    seq1= random.choices(range(65, 91), k=3) + random.choices(range(48, 58), k=4) + random.choices(range(65, 91), k=3)

    #Indices in ASCII (from A->Z)
    seq2= random.choices(range(65, 91), k=1)

    # Plan applicable in ASCII (3letters+ "315"+ 3numbers)
    seq3= random.choices(range(65, 91), k=3) + [51,49, 53] + random.choices(range(48, 58), k=3)

    # Dates (jj/mm/aaaa) 2010 <aaaa< 2020
    seq4= random.choices(range(48, 58), k=2)+ ["/"]+ random.choices(range(48, 58), k=2)+ ["/"]+ [50, 48]+ random.choices(range(49, 51), k=1)+ random.choices(range(48, 58), k=1)

    # Dates (jj/mm/aa) 10 <aa< 20
    seq5= random.choices(range(48, 58), k=2)+ ["/"]+ random.choices(range(48, 58), k=2)+ ["/"]+ random.choices(range(49, 51), k=1)+ random.choices(range(48, 58), k=1)

    # Plan applicable in ASCII (3letters+ "313"+ 3 numbers)
    seq6= random.choices(range(65, 91), k=3) + [51,49, 51] + random.choices(range(48, 58), k=3)
    
    # Cable Name in ASCII (3letters+ 4numbers + 3letters+ /+ 1letter)
    seq7= random.choices(range(65, 91), k=3) + random.choices(range(48, 58), k=4) + random.choices(range(65, 91), k=3)+ ["/"]+ random.choices(range(49, 51), k=1)

#     seq8= random.choices(range(65, 91), k=3) + random.choices(range(48, 58), k=4) + random.choices(range(65, 91), k=3)+ ["/"]+ random.choices(range(49, 51), k=1)
    
    for seq in [seq1, seq2, seq3, seq4, seq5, seq6, seq7]:
        string = ascii_seq_to_string(seq)
        img_id+=1

        try:
            margin= 30
            string= string.replace(" ", "")
            string_img= generation.generate_sentence_image(string)
            
        except:
            continue

        string_img= preprocessing.add_border(img=string_img, bordersize= margin)


        #Save the generated image 
        cv2.imwrite(os.path.join(saving_dir, f"Generated_img_{img_id}.jpg"), string_img)
        height, width= string_img.shape[:2]
        
        #Generate the COCO annotations: image filename, regions coordinates, ...
        img_dict= {"file_name": f"Generated_img_{img_id}.jpg", "id": img_id,
                   "width": width, "height": height, 
                   "date_captured": "", "flickr_url": "", "license": 0, "coco_url": ""}
        coco_dict["images"].append(img_dict)

        for letter in range(len(string)):
            letter_id+=1

            h= generation.charH
            w= generation.charW
            (x0, y0)= ( letter*w+margin, margin )
            (x1, y1)= ( (letter+1)*w + margin, margin )
            (x2, y2)= ( (letter+1)*w + margin, h+ margin )
            (x3, y3)= ( letter*w+margin , h+ margin )
            coco_dict["annotations"].append({'image_id':img_id, 
                                             "segmentation": [[x0, y0, x1, y1,
                                                               x2, y2, x3, y3
                                                              ]],
                                             "bbox": [x0, y0, w, h],
#                                              "bbox_mode": BoxMode.XYXY_ABS,
                                              "category_id": 1,
                                             "id": letter_id,
                                             "area": w*h,
                                             "iscrowd": 0
                                            })

# Save the COCO annotations in the json file  
with open(os.path.join(str(Path(saving_dir).parent), json_file), "w") as f:
            json.dump(coco_dict, f)